# Tune Hyperparams with AML

In ML, models are trained to predict unknown labels for new data based on correlations between known labels and features in the training data. Depending on the algo, you may need to specify hyperparams to configure how the model is trained. 
E.g. logist regression uses regularisation rate hyperparam to counteract overfitting. 

remember:
- parameters are derived from the training data by the model
- hyperparameters are not 

Hyperparameter tuning happens when you are able to train the same algorithm and training data, but different hyperparameter values. The output model is then evaluated against one another to check the performance metric for which you want to optimise, and then allowing you to select the best model.

In AML, you can do this in an experiment type called **hyperdrive run**. This initiates a child run for each hyperparameter combination that needs to be tested. Each child run uses a training script with parameterised hyperparameter values that trains the model, and logs the target performance metric.

## Define a Search Space

Combination of hyperparameter values tried during tuning is known as **search space**. 

You can specify your hyperparameters as:
- a value
- values:
    - list - choice([10, 20, 30])
    - range - choice(range([1,10]))
    - comma-separated values - choice(30,50,100)
- distribution:
    - qnormal
    - quniform
    - qlognormal
    - qloguniform

To define a search space, you have to create a dictionary with the appropriate parameters. e.g.
```python
from azureml.train.hyperdrive import choice, normal

param_space={
    '--batch_size':choice(16,32,64),
    '--learning_rate': normal(10,3)
}
```

## Types of sampling

Types of sampling:
- Grid
- Random
- Bayesian

#### Grid Sampling 
This is only used when all hyperparams are discrete, and when you want to try all possible combinations. e.g.
```python
from azureml.train.hyperdrive import GridParameterSamping, choice
param_space = {
    '--batch_size': choice(16, 32, 64),
    '--learning_rate': choice(0.01, 0.1, 1.0)
}
param_sampling = GridParameterSampling(param_space)
```

#### Random Sampling 
Used to randomly select values for each hyperparameter. This can be a mix of discrete and continuous values. e.g.
```python
from azureml.train.hyperdrive import RandomParameterSampling, choice, normal

param_space = {
    '--batch_size': choice(16,32,64),
    '--learning_rate': normal(10,3)
}
```

#### Bayesian Sampling 
choose hyperparam values based on Bayesian optimsation algorithms. This tries to select parameter combinations that will result in improved performance based on previous selections. You can only use choice, uniform or quniform params with bayesian sampling. You can also not use early-termination policy. e.g.
```python 
param_space = {
    '--batch_size': choice(16,32,64),
    '--learning_rate': uniform(0.5, 0.1)
}
```

## Early Termination

If you ahve a large hyperparam search space, it may take too long to try all possible combinations. Normally, you set a max number of iterations, but this can still lead to too many runs that dont result in better model.

To stop wasting time, you can set an early termination policy. It will abandon runs that are unlikely to produce a better results than previous runs. 

It is evaluated at **evaluation_interval**, this is a parameter that you specify, based on each time the target perfomance metric is logged. You can also set ***delay_evaluation*** param to avoid evaluating policy until a minimum number of iterations have run.

Early termination is particularly helpful for Deep Neural Nets, as they are trained iteratively over a number of epochs.

Types of policies:
- Bandit
- Median Stopping
- Truncation

#### Bandit policy
use this to stop a run if the target performance metric is underperforming the best run so far by a specified margin.
```python
from azureml.train.hyperdrive import BanditPolicy

early_termination_policy = BanditPolicy(
    slack_amount = 0.2, 
    evaluation_interval = 1,
    delay_evaluation = 5
)
- delay_evaluation: when to start. In this case, it will run after the first five
- slack_amount: when to abandon. It will abandon runs when the reported taret metric is 0.2 worse than the best performing run

#### Median Stopping policy
This abandons runs where the target performance metric is worse than the median of the running averages for all runs. 
```python
from azureml.train.hyperdrive import MedianStoppingPolicy

early_termination_policy = MedianStoppingPolicy(
    evaluation_interval=1,
    delay_evaluation=5
)
```

#### Truncation Selection policy
This cancels the lowest performing X% of runs at each evaluation interval based on the truncation_percentage value you specified.
```python
from azureml.train.hyperdrive import TruncationSelectionPolicy

early_termination_policy = TruncationSelectionPolicy(
    truncation_percentage=10,
    evaluation_interval=1,
    delay_evaluation=5
)
```

## Running a hyperparam tuning experiment

In AML, you can tune hyperparameters by running a ***hyperdrive*** experiment.

To run a hyperdrive experiment, you **must create a training script** and:
- include an argument for each hyperparameter you want to vary
- log the target performance metric. This will allow the hyperdrive run to evaluate the child runs it initiaties, and identify the best one that produces the best performing model

sample:

```python
#libraries go here 

# Get regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01)
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# get and split data

# Train a logistic regression model with the reg hyperparameter
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate and log accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# Save the trained model
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model.pkl')

run.complete()

```

### Configuring and running a hyperdrive experiment

To prepare a hyperdrive exp, you have to use a HyperDriveConfig object to configure the experiment.
```python
from azureml.core import Experiment 
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal 

# assumes ws, sklearn_estimator and param_sampling are already defined
hyperdrive = HyperDriveConfig(
    estimator = sklearn_estimator, # above
    hyperparameter_sampling = param_sampling, #above
    policy = None, # can be from above 
    primary_metric_name= 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 6,
    max_concurrent_runs = 4
)
experiment = Experiment(ws, 'hyperdrive_training')
hyperdrive_run = experiment.submit(config=hyperdrive)
```

### Monitoring & Reviewing run 

You can monitor an experiment the usual way:
- In studio or using RunDetails(run) widget

The experiment will create a child run for each hyperparam combination to be tried, and you can review the logged metrics by:
```python
for child_run in run.get_children():
    print(child_run.id, child_run.get_metrics())
```

To retreive the best performing run, you can:
```python
best_run = hyperdrive_run.get_best_run_by_primary_metric()
```